<img src="https://assets.zyrosite.com/d9510y2R1JHwG05k/covary-1-dWxO42PGWjI8Kp03.png" height="200" align="right" style="height:340px">

#Covary v2.0: Deep learning-based phylogenetic reconstruction using [TIPs-VF](https://www.biorxiv.org/content/10.1101/2025.02.15.637782v1)

Covary is a deep learning framework for phylogenetic reconstruction that leverages the TIPs-VF genetic representation. TIPs-VF belongs to the Translator-Interpreter Pre-seeding (TIPs) family, a set of encoding schemes designed to enhance the numerical representation of genetic sequences for machine learning applications. By integrating Keras-based neural network architectures, Covary provides an efficient and scalable approach to reconstructing phylogenetic trees.

---

**Versions:**

*Public release*
* [2.1](https://colab.research.google.com/drive/1wZ0hmDZzAlQkHALUbrN0txkUVCJssReZ?usp=sharing) – Adds user-defined plot customization and introduces filtering of sequence entries containing invalid characters, enabling faster and more reliable project implementation.
* [2.0](https://colab.research.google.com/drive/1_DyU37rW-YZ8sxUP_BhnoUYFjiv1NNK0?usp=sharing) – Introduces codon normalization, resolving inconsistencies in the relationship between input sequences and open reading frames or protein translations within unaligned datasets.

*Pre-release*
* [1.3](https://colab.research.google.com/drive/1y9-0pyWNG5SlAUieGp7-ZSfQ3qN6E7VB?usp=sharing) – Optimizes the perplexity parameter for t-SNE, enabling reliable analysis of datasets with limited sample sizes.
* [1.2](https://colab.research.google.com/drive/1jwwN_OKAspYaYjDoyoYfivoGf_QF1kHY?usp=sharing) – Reduces sequence ambiguity, improving support for datasets containing unprocessed or ambiguous genetic sequences.
*   [1.1](https://colab.research.google.com/drive/1EkTk7vaBUqCiQFkH1LtK5RfjQW7p0i2D?usp=sharing) – Enhances memory efficiency to prevent session crashes caused by RAM overload.
*   [1.0](https://colab.research.google.com/drive/14DHrXhgHjL7ieUILnlpl1mzqo6rFY7C4?usp=sharing) – Initial release (unsupported)

##Prepare your sequence data, then hit `Runtime` -> `Run all`
Fields marked with ⚠️ may require your attention or input, please don't collapse them while running Covary

### Step 1. ⚠️ Upload your data

In [ ]:
import os
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]
var_name = "input_seq.fasta"
os.rename(filename, var_name)

###Step 2. Install dependencies

In [ ]:
# This will install the ff packages/modules
import os
import sys
import re
import math
from itertools import product
!pip install umap-learn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from tensorflow.keras import layers, models
from scipy.cluster.hierarchy import linkage, dendrogram
import seaborn as sns
from scipy.spatial.distance import pdist, squareform
import zipfile
from datetime import datetime
import sys, builtins

###Step 3. QC Check

In [ ]:
# Pre-process the data with whitespace remover + 'clean_seq.py' from https://github.com/mahvin92/TIPs-VF/blob/main/pre-processing/clean_seq.py

def replace_non_atcg_remove_whitespace(file_name, wrap_width=None):
    def wrap_seq(seq, width):
        return [seq[i:i+width] for i in range(0, len(seq), width)]

    with open(file_name, 'r') as fh:
        lines = fh.readlines()

    with open(file_name, 'w') as fh:
        for line in lines:
            if line.startswith('>'):
                fh.write(line.rstrip() + '\n')
            else:
                # Remove ALL whitespace (including internal spaces/tabs/newlines),
                # then uppercase.
                seq = re.sub(r'\s+', '', line)
                seq = seq.upper()

                if seq == '':
                    continue

                # Replace non-ATCG with 'N'
                cleaned = ''.join([ch if ch in 'ATCG' else 'N' for ch in seq])

                if wrap_width and isinstance(wrap_width, int) and wrap_width > 0:
                    for chunk in wrap_seq(cleaned, wrap_width):
                        fh.write(chunk + '\n')
                else:
                    fh.write(cleaned + '\n')


# -----------------------------
# Run
# -----------------------------
file_name = "input_seq.fasta"
replace_non_atcg_remove_whitespace(file_name, wrap_width=60)  # change value to set wrap number

###Step 4. Covary encoding

In [ ]:
# Clone Covary-encoder
!git clone https://github.com/mahvin92/Covary-encoder.git
%cd Covary-encoder
!git sparse-checkout init --cone
!git sparse-checkout set Active README.md

# Reconstruct runtime
%cd Active
os.mkdir("codeenigma_runtime")
!mv /content/Covary-encoder/Active/__init__.py /content/Covary-encoder/Active/codeenigma_runtime
!mv /content/input_seq.fasta /content/Covary-encoder/
print("Your input file has been moved to the Covary-encoder directory for encoding")
%cd /content/Covary-encoder/
!pip install /content/Covary-encoder/Active/codeenigma_runtime-*.whl

###Step 5. Data extraction ⚠️

In [ ]:
# Run Covary-encoder
sys.path.append("/content/Covary-encoder/Active")
builtins.exit = sys.exit
import Covary_encoder

###Step 6. Deep learning

In [ ]:
# Start DL by Load dataset
file_path = "seq_TIPs-encoded.tsv" # This can be downloaded from the file path directory
data = pd.read_csv(file_path, sep='\t')

print(data.head())

# Extract features
def process_column(col):
    return np.array([float(x) for x in col.split(',')])

# Parse all cos_sim and theta columns
X_cos_sim_1 = data['cos_sim_1'].apply(process_column)
X_cos_sim_2 = data['cos_sim_2'].apply(process_column)
X_cos_sim_3 = data['cos_sim_3'].apply(process_column)

X_theta_s_1 = data['theta_s_1'].apply(process_column)
X_theta_s_2 = data['theta_s_2'].apply(process_column)
X_theta_s_3 = data['theta_s_3'].apply(process_column)

# Compute averages index-wise
X_avg_cos_sim = [
    (x1 + x2 + x3) / 3.0
    for x1, x2, x3 in zip(X_cos_sim_1, X_cos_sim_2, X_cos_sim_3)
]

X_avg_theta = [
    (t1 + t2 + t3) / 3.0
    for t1, t2, t3 in zip(X_theta_s_1, X_theta_s_2, X_theta_s_3)
]

# Final combined feature vector
X_combined = np.array([
    np.concatenate([avg_cos, avg_theta])
    for avg_cos, avg_theta in zip(X_avg_cos_sim, X_avg_theta)
])

print(f"Shape of the data: {X_combined.shape}")

# Data scaling
X_scaled = StandardScaler().fit_transform(X_combined)

# Get the labels (assuming it's available)
labels = data['Gene_name']  # Replace with your actual label column name

label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Build the Autoencoder-like Neural Network model
def build_autoencoder(input_dim, embedding_dim=10):
    encoder_input = layers.Input(shape=(input_dim,))  # Define input shape explicitly
    x = layers.Dense(128, activation='relu')(encoder_input)
    x = layers.Dense(64, activation='relu')(x)
    encoded = layers.Dense(embedding_dim, activation='linear')(x)  # Embedding layer output dimension

    x = layers.Dense(64, activation='relu')(encoded)
    x = layers.Dense(128, activation='relu')(x)
    decoded = layers.Dense(input_dim, activation='linear')(x)  # Reconstructing to input_dim

    # Autoencoder (encoder + decoder)
    autoencoder = models.Model(encoder_input, decoded)

    return autoencoder

# Initialize the model
autoencoder_model = build_autoencoder(X_scaled.shape[1], embedding_dim=10)

# Compile the model with Mean Squared Error loss for reconstruction
autoencoder_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model on the input data to learn the embedding representation
autoencoder_model.fit(X_scaled, X_scaled, epochs=50, batch_size=32, verbose=1)

# Extract the embeddings (output of the encoder)
encoder = models.Model(inputs=autoencoder_model.input, outputs=autoencoder_model.layers[3].output)
embeddings = encoder.predict(X_scaled)

###Step 7. Scoring and analysis

In [ ]:
# Get vector embeddings for:
# PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(embeddings)

# t-SNE
n_samples = embeddings.shape[0]
safe_perplexity = max(2, min(30, (n_samples - 1) // 2))  # auto-adjust
tsne = TSNE(n_components=2, random_state=42, perplexity=safe_perplexity)
tsne_result = tsne.fit_transform(embeddings)

# UMAP
umap_model = umap.UMAP(n_components=2, random_state=42)
umap_result = umap_model.fit_transform(embeddings)

In [ ]:
# Make results directory
%cd /content/
os.makedirs("covary_results", exist_ok=True)
cool_warm = plt.cm.coolwarm
cool = plt.cm.cool
warm = plt.cm.autumn

# -----------------------------
# Helpers
# -----------------------------
def save_plot(fig, filename):
    fig.savefig(f"covary_results/{filename}", dpi=300, bbox_inches="tight")
    plt.close(fig)

def save_embeddings(embeddings, filename, labels=None):
    """Save embeddings with headers and optional row labels."""
    df = pd.DataFrame(embeddings, columns=[f"Dim{i+1}" for i in range(embeddings.shape[1])])
    if labels is not None:
        df.insert(0, "Label", labels)
    df.to_csv(f"covary_results/{filename}", sep="\t", index_label="Sample")

def save_distances(distances, filename):
    """Save pairwise distance matrix with row/col headers."""
    n = distances.shape[0]
    labels = [f"Sample{i+1}" for i in range(n)]
    df = pd.DataFrame(distances, index=labels, columns=labels)
    df.to_csv(f"covary_results/{filename}", sep="\t")

def save_linkage(linkage_matrix, filename):
    """Save linkage matrix with informative headers."""
    df = pd.DataFrame(linkage_matrix,
                      columns=["Cluster1", "Cluster2", "Distance", "SampleCount"])
    df.to_csv(f"covary_results/{filename}", sep="\t", index=False)

# -----------------------------
# 1. Embedding plots
# -----------------------------
def plot_embeddings_and_save(embeddings, encoded_labels, gene_names, title, fname):
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    scatter = ax.scatter(embeddings[:, 0], embeddings[:, 1],
                         c=encoded_labels, cmap=plt.cm.coolwarm, alpha=0.5, s=100)
    ax.set_title(title)

    for i, gene_name in enumerate(gene_names):
        ax.annotate(
            gene_name,
            xy=(embeddings[i, 0], embeddings[i, 1]),
            xytext=(5, 5),
            textcoords='offset points',
            fontsize=5,
            color='black',
            alpha=0,       # adjust transparency (0 = fully transparent, 1 = fully opaque)
            arrowprops=dict(arrowstyle='-', lw=0.5, color='gray', alpha=0) # arrows also semi-transparent
        )

    plt.colorbar(scatter, ax=ax, label='Sequence entry/group')
    save_plot(fig, fname)

# Save embeddings arrays with headers
save_embeddings(pca_result, "pca_embeddings.tsv", labels=labels)
save_embeddings(tsne_result, "tsne_embeddings.tsv", labels=labels)
save_embeddings(umap_result, "umap_embeddings.tsv", labels=labels)

plot_embeddings_and_save(pca_result, encoded_labels, labels.tolist(), "PCA Embeddings", "pca_embeddings.png")
plot_embeddings_and_save(tsne_result, encoded_labels, labels.tolist(), "t-SNE Embeddings", "tsne_embeddings.png")
plot_embeddings_and_save(umap_result, encoded_labels, labels.tolist(), "UMAP Embeddings", "umap_embeddings.png")

# -----------------------------
# 2. Pairwise distances
# -----------------------------
def plot_heatmap_and_save(matrix, title, fname, gene_names):
    """Plot heatmap with gene names on both axes."""
    fig = plt.figure(figsize=(25, 20))
    sns.heatmap(matrix, cmap=plt.cm.coolwarm, cbar=True,
                xticklabels=gene_names, yticklabels=gene_names)
    plt.title(title)
    plt.xlabel('Sample ID')
    plt.ylabel('Sample ID')
    save_plot(fig, fname)

pca_distances = squareform(pdist(pca_result, metric='euclidean'))
umap_distances = squareform(pdist(umap_result, metric='euclidean'))
tsne_distances = squareform(pdist(tsne_result, metric='euclidean'))

save_distances(pca_distances, "pca_distances.tsv")
save_distances(umap_distances, "umap_distances.tsv")
save_distances(tsne_distances, "tsne_distances.tsv")

plot_heatmap_and_save(pca_distances, "Heatmap of PCA Embedding Distances", "pca_heatmap.png", labels.tolist())
plot_heatmap_and_save(umap_distances, "Heatmap of UMAP Embedding Distances", "umap_heatmap.png", labels.tolist())
plot_heatmap_and_save(tsne_distances, "Heatmap of t-SNE Embedding Distances", "tsne_heatmap.png", labels.tolist())

# -----------------------------
# 3. Dendrograms
# -----------------------------
def plot_dendrogram_and_save(embeddings, method, title, fname, gene_names):

    fig = plt.figure(figsize=(25, 50))
    linkage_matrix = linkage(embeddings, method=method)
    dendrogram(linkage_matrix,
               leaf_rotation=0,
               leaf_font_size=5,
               orientation="left",
               labels=gene_names)
    plt.title(title)
    plt.ylabel('Sample ID')
    plt.xlabel('Distance')
    save_plot(fig, fname)
    save_linkage(linkage_matrix, fname.replace(".png", ".tsv"))

methods = ["ward", "complete", "average", "single"]

for method in methods:
    plot_dendrogram_and_save(pca_result, method,
                             f"PCA Dendrogram ({method})",
                             f"pca_dendrogram_{method}.png",
                             labels.tolist())
    plot_dendrogram_and_save(umap_result, method,
                             f"UMAP Dendrogram ({method})",
                             f"umap_dendrogram_{method}.png",
                             labels.tolist())
    plot_dendrogram_and_save(tsne_result, method,
                             f"t-SNE Dendrogram ({method})",
                             f"tsne_dendrogram_{method}.png",
                             labels.tolist())


# -----------------------------
# 4. Zip and download results
# -----------------------------
date_str = datetime.now().strftime("%Y%m%d")
zip_filename = f"Covary results_{date_str}.zip"

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files_in_dir in os.walk("covary_results"):
        for file in files_in_dir:
            # keep only .png and .tsv (but exclude the heavy seq_TIPs-encoded.tsv if it exists)
            if file.endswith(".png") or (file.endswith(".tsv") and "seq_TIPs-encoded" not in file):
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, "covary_results"))

###Step 8. ⚠️ Download results

In [ ]:
files.download(zip_filename)
print("Covary has finished running")

#Instructions

**Quick start**

1. Press ```"Runtime" -> "Run all".```
2. Upload your FASTA-formatted genetic sequence(s) in **Step 1** (recommended file types: .txt or .fasta).
3. Fields marked with ⚠️ may require your attention or input. Do not collapse these fields while running Covary.
4. The pipeline consists of 8 steps. The currently running step is indicated by a spinning circle with a stop sign.


**Best Practices**

* Ensure that all sequences cover the same genomic region. For example, in taxonomic analysis using bacterial 16S rRNA, if one entry begins at base position TSS +1, all other 16S rRNA sequences should also start at or close to that position.

* Before interpreting dendrograms, first inspect the generated vector embeddings. Evaluate clustering results and determine which dimensionality reduction method provides the clearest resolution of relationships between your sequences. Examine pairwise distances of the vector embeddings when necessary.

* Based on the chosen reduction method, identify which linkage method (Ward, Average, Complete, Single) provides the best subgrouping of your data points.


**Sequence representtion**

Name your file as ```input_seq.fasta```, a critical requirement for representation. Input genetic sequences are represented using [TIPs-VF](https://doi.org/10.1101/2025.02.15.637782) and processed by a deep learning model built on Keras. The resulting TIPs-encoded sequences can be downloaded from the file browser as ```seq_TIPs-encoded.tsv```. Vector embeddings generated by PCA, t-SNE, and UMAP are automatically included in the zipped results (in TSV format).


**Phylogenetic reconstruction**

Covary applies multiple linkage methods (Ward, Average, Complete, Single) to infer and compare relationships among the represented input sequences. Deep learning enables recognition of unique patterns and resolution of sequence differences. A phylogenetic tree (without branch length) is reconstructed and inferred based on denodrogram clustering (similar to a cladogram-type visualization).


**Result zip file contents**

1. **Vector embeddings**: Numerical data in ```.tsv``` format for PCA, t-SNE, and UMAP.
2. **Vector embedding plots**: Scatter plots of Dim-1 *vs.* Dim-2, labeled by sequence entry. A color gradient (z-score) standardizes sequence indices regardless of dataset size.
3. **Heatmap (pairwise distance plot)**: Euclidean pairwise distance plots, provided alongside each reduction analysis.
4. **Dendrogram linkages**: Numerical data in ```.tsv``` format for linkage analyses (Ward, Average, Complete, Single) across PCA, t-SNE, and UMAP.
5. **Dendrogram plots**: Visualizations showing distances (x-axis) and sequence indices (y-axis) across different linkage methods and reduction analyses.


**Troubleshooting**

* Check that the runtime type is set to GPU at ```"Runtime" -> "Change runtime type"```.
* Try to restart the session ```"Runtime" -> "Factory reset runtime"```.
* Check your input sequence for the presence of invalid sequence characters or white spaces. Note that Covary_encoder can only represent A, T, C, G, N sequences to reduce ambiguity factors that may influence the deep learning results. Limit the use of non-conventional DNA sequences, as much as possible, and resolve sequence data using a reference assembly. *Newer versions of Covary (e.g., v.2.0 and above) have added QC check step that removes whitespaces and filter param that is set to ignore/remove data/sequence entry containing invalid sequences prior to representation; ```see Steps 1 and 3```)*
* Pre-process your data, when needed. Most problems with non-ATCGN in Covary_encoder can be fixed by running your data first in the 'clean_seq.py' that can be downloaded from TIPs-VF [GitHub](https://github.com/mahvin92/TIPs-VF/tree/main/pre-processing). *Newer versions of Covary (e.g., v.2.0 and above) have incorporated this workflow; ```see Steps 1 and 3```)*
* If download failed to start, rerun ```Step 8```.


**Bug**

If you encounter any bugs, please open a ticket [here](https://github.com/mahvin92/Covary/issues/new) or request a support [here](https://covary.chordexbio.com).

**Usage**

Covary can be applied to a variety of studies [see performance validations](https://covary.chordexbio.com):
1. Clade or strain reconstruction in viral infections
2. Taxonomic species evolution
3. Clonal and driver mutation tracking in tumor evolution
4. Genetic relationship and divergence studies

**Funding**

```None```


**License**

[Read here](https://github.com/mahvin92/Covary/blob/main/LICENSE)

**Footnote**

Covary is powered by [TIPs](https://tips.chordexbio.com/) and [ChordexBio](https://chordexbio.com/), made with Python, and tested using Google Colab ❤️